In [13]:
import time
from selenium import webdriver
from datetime import datetime

# CONFIGURATION
website = #website
email = #email
pword = #password
people_count = 30 # over-shoots the number of chart elements, this forgoes trying to figure out how many people should be on the list
day_count = 10 # how many days' worth of data are being scraped, going backwards in time
log_path = r"C:\Users\Alex\Desktop\python\countit_logs"

# XPATH LOCATIONS
login_xpath = '/html/body/div[2]/div[2]/div[1]/div/div/div[3]/div/form/div[3]/div[1]/button'
timing_toggle_xpath = '/html/body/div[2]/div[2]/div[1]/div/div[1]/div[1]/div/div[3]/div/div/div[2]/div[1]/div[1]/div/div[1]'
timing_option_xpath = '/html/body/div[2]/div[2]/div[1]/div/div[1]/div[1]/div/div[3]/div/div/div[2]/div[1]/div[1]/div/div[1]/ul/li[3]'
points_toggle_xpath = '/html/body/div[2]/div[2]/div[1]/div/div[1]/div[1]/div/div[3]/div/div/div[2]/div[1]/div[3]/div/div'
points_option_xpath = '/html/body/div[2]/div[2]/div[1]/div/div[1]/div[1]/div/div[3]/div/div/div[2]/div[1]/div[3]/div/ul/li[4]/a'
chart_xpath = '/html/body/div[2]/div[2]/div[1]/div/div[1]/div[3]/div/div[2]/div[3]/div/div[2]/div/div'
date_xpath = '/html/body/div[2]/div[2]/div[1]/div/div[1]/div[1]/div/div[3]/div/div/div[2]/div[1]/div[1]/div/div[2]/div/div[2]/div/div'
date_back_xpath = '/html/body/div[2]/div[2]/div[1]/div/div[1]/div[1]/div/div[3]/div/div/div[2]/div[1]/div[1]/div/div[2]/div/div[1]/i'

# FUNCTIONS
def date_converter(date_value):
    # converts scraped date into MM/DD/YYYY with the current year, so "APR 5" becomes "04/05/2021"
    month, day = date_value.strip().split()
    month_dict = {'JAN':'01','FEB':'02','MAR':'03',
                  'APR':'04','MAY':'05','JUN':'06',
                  'JUL':'07','AUG':'08','SEP':'09',
                  'OCT':'10','NOV':'11','DEC':'12'}
    day = ('0' + day)[-2:]
    month = month_dict[month]
    year = datetime.now().year
    datestamp = f"{month}/{day}/{year}"
    return datestamp

def scrape_chart(chart_xpath, people_count):
    # Iterates through the chart and reads the text for each bar
    result_list = []
    for i in range(people_count):
        try:
            chart_element = driver.find_element_by_xpath(chart_xpath + f"/div[{i}]")
            result_list.append(chart_element.text)
        except:
            pass
    return result_list

def scrape_parser(scraped_dict):
    # This reads the raw text scraped from the website and builds a clean "Day,Name,Points" string for the data base
    parsed_entries = []
    for date, elements in scraped_dict.items():
        for person in elements:
            if person.split('\n')[-1].isnumeric(): # the rows we want end with a linebreak followed by a number, e.g. '\n135'
                name, points = person[person.index('.') + 1 :].split('\n') # strips out the leading rank indicator, e.g. '3. '
                name = name.encode('ascii', 'ignore').decode('ascii').strip('.').strip() # cleans up the name, also removing emojis and weird chars
                entry = f"{date},{name},{points}" # this our entry into the data file
                parsed_entries.append(entry)
    return parsed_entries

def write_parsed_log(log_path, parsed_entries):
    # Writes a new log file with the parsed data
    file_name = 'countit_data_' + datetime.now().strftime("%Y%m%d%H%M%S") + '.txt' # file name has timestamp in the name make it unique
    file_path = log_path + "\\" + file_name
    with open(file_path, "w") as file:
        for entry in parsed_entries:
            file.write(entry + '\n')
        file.close()

# MAIN
if __name__ == '__main__':
    driver = webdriver.Chrome()
    driver.maximize_window() #xpath behaves weirdly for small windows
    driver.get(website)

    # Log in using credentials in config
    driver.find_element_by_id('email').send_keys(email)
    driver.find_element_by_id('password').send_keys(pword)
    time.sleep(1) # wait for login button to appear
    driver.find_element_by_xpath(login_xpath).click()
    time.sleep(2) # let new page load

    # Toggle options to be DAILY points
    driver.find_element_by_xpath(timing_toggle_xpath).click()
    time.sleep(1) # wait for drop-down to load
    driver.find_element_by_xpath(timing_option_xpath).click()

    # Toggle options to be TOTAL points
    driver.find_element_by_xpath(points_toggle_xpath).click()
    time.sleep(1) # wait for drop-down to load
    driver.find_element_by_xpath(points_option_xpath).click()

    # Scrape the data for the past few days and put it in a dict by day
    scraped_dict = {}
    for _ in range(day_count):
        time.sleep(1) # wait to let data load
        
        # create a datestamp for the current displayed data
        date_value = driver.find_element_by_xpath(date_xpath)
        datestamp = date_converter(date_value.text) 
        
        scraped_dict[datestamp] = scrape_chart(chart_xpath, people_count) # scrape the data and enter into dict by date

        driver.find_element_by_xpath(date_back_xpath).click() # toggle to the previous day
    
    # Parse the raw text and create a new log file for it
    parsed_entries = scrape_parser(scraped_dict)
    driver.quit()
    write_parsed_log(log_path, parsed_entries)    

In [14]:
def build_db(db_name, table_list, print_log = False):
    import sqlite3
    connection = sqlite3.connect(db_name)
    cursor = connection.cursor()
    table_check = [str(row).strip('(').strip(')').strip(',').strip(r"'") for row in cursor.execute('''SELECT name FROM sqlite_master WHERE type="table"''')]
    log = [f'Table check for {db_name}:']
    table_count = 0
    for table, create in table_list.items():
        if table in table_check:
            log.append(f"Already exists: {table}")
            table_count += 1
        else:
            try:
                cursor.execute(create)
            except:
                log.append(f"Failed to add: {table}")
            else:
                log.append(f"Added: {table}")
                table_count += 1
    log.append(f"{table_count} of {len(table_list)} tables accounted for.")
    if print_log is True:
        for line in log:
            print(line)
    cursor.close()
    connection.close()

In [25]:
db_name = 'countit.db'
table_list = {'daily_points':'''CREATE TABLE daily_points
                    (p_ID INTEGER PRIMARY KEY AUTOINCREMENT,
                    alias TEXT NOT NULL,
                    created_date TEXT NOT NULL,
                    date TEXT NOT NULL,
                    points TEXT)''',
              'people':'''CREATE TABLE people
                    (p_ID INTEGER PRIMARY KEY AUTOINCREMENT,
                    alias TEXT NOT NULL,
                    name TEXT NOT NULL)'''}
build_db(db_name, table_list, print_log = False)

In [17]:
import sqlite3
log_file = "countit_data_20210318194755.txt"
connection = sqlite3.connect(db_name)
cursor = connection.cursor()
with open(r"C:\Users\Alex\Desktop\python\countit_logs\\" + log_file) as file:
    while True:
        entry = file.readline()
        created_date = log_file.strip('.txt').split('_')[2]
        if entry:
            date, alias, points = entry.strip('\n').split(',')
            params = (alias, created_date, date, points)
            cursor.execute("INSERT INTO daily_points VALUES (NULL, ?, ?, ?, ?)", params)
        else:
            break
    file.close()
cursor.close()
connection.close()

In [24]:
db_name = 'countit.db'
connection = sqlite3.connect(db_name)
cursor = connection.cursor()
cursor.execute('''SELECT * FROM daily_points WHERE alias = "Alex B"''')
rows = cursor.fetchall()
print(rows)
cursor.close()
connection.close()

[]


('Chris', '20210318004051', '03/11/2021', '12')